### [1]모듈 로딩 및 처리

In [1]:
# 모듈 로딩 
import torch # torch 기본 모듈로 텐서 기본 함수들 
import torch.nn.functional as F # pytorch 인공 신경망 관련 함수들 
from sklearn.datasets import fetch_openml
import numpy as np
import torch
import torch.nn as nn # linear Regression 기능의 클래스 Linear
import torch.nn.functional as F # 손실함수를 위함
import torch.optim as optim # 최적화를 위함
from torch.utils.data import Dataset,DataLoader
import pandas as pd

### [2] 데이터 준비

In [2]:
from sklearn.datasets import fetch_openml 
import pandas as pd
db_name = 'Fashion-MNIST'
fashion_data = fetch_openml(name=db_name, parser='auto',as_frame=True)


In [3]:
### ====> 데이터 확인
feature=fashion_data.data
target=fashion_data.target

In [4]:
# feature=np.array(feature)
# target=np.array(target)

In [5]:
# 피쳐와 타겟 개수 확인
print(f'Feature: {feature.shape},{feature.ndim}D')
print(f'Target: {target.shape},{target.ndim}D')  #원핫 인코더 쓸때 확인

Feature: (70000, 784),2D
Target: (70000,),1D


In [6]:
# data= 피쳐(개수 784개)
fashion_data.data

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,119,114,130,76,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,22,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,33,96,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
69996,0,0,0,0,0,0,0,0,0,31,...,0,0,0,0,0,0,0,0,0,0
69997,0,0,0,0,0,0,0,0,0,0,...,27,0,0,0,0,0,0,0,0,0
69998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
class DLDataset(Dataset):
    def __init__(self,x_data,y_data):
        super().__init__()
        #x,y 데이터 ==> ndarray
        x_data=x_data.values if isinstance(x_data,pd.DataFrame) else x_data #values는 넘파이로 바꾸어 주는 친구
        y_data=y_data.values if isinstance(y_data,pd.DataFrame) else y_data
        
        #ndarray ==> tensor(데이터 타입이 뭐가 들어올지 몰라서 tensor를 씀)
        self.feature=torch.FloatTensor(x_data)
        self.target=torch.LongTensor(y_data)  #0,1,2(정수)로 라벨 인코더를 사용해 바꿔놨음
        print('[target & target SHAPE]',self.target.shape,self.target.ndim)
        
    #데이터셋의 갯수 체크 함수
    def __len__(self):
        return len(self.target)
        #return self.target.shape[0]
    
    #특정 인덱스 데이터+라벨 반환 콜백함수(callback function)
    def __getitem__(self,index):
        return self.feature[index],self.target[index]  #튜플로 반환 

[2-2]데이터셋 인스턴스 생성

In [8]:
print(feature), print(type(feature))

       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       1       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0      33   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
69995       0       0       0       0       0       0       0       0       0   
69996       0       0       0       0       0       0       0       0       0   
69997       0       0       0       0       0       0       0       0       0   
69998       0       0       0       0       0       0       0       0       0   
69999       0       0       0       0       0       0       0       0       0   

       pixel10  ...  pixel7

(None, None)

In [9]:
print(target), print(type(target))

0        9
1        0
2        0
3        3
4        0
        ..
69995    9
69996    1
69997    8
69998    1
69999    5
Name: class, Length: 70000, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']
<class 'pandas.core.series.Series'>


(None, None)

In [10]:
## 데이터를 feature와 라벨로 분리
featureDF=feature
targetDF=target.to_frame()  #데이터 프레임화 해서 2차원으로 변경

print(f'featureDF=> {featureDF.shape},{featureDF.ndim}D')
print(f'targetDF=> {targetDF.shape},{targetDF.ndim}D')

featureDF=> (70000, 784),2D
targetDF=> (70000, 1),2D


In [11]:
targetDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   class   70000 non-null  category
dtypes: category(1)
memory usage: 68.9 KB


In [12]:
type(feature)

pandas.core.frame.DataFrame

In [13]:
targetDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   class   70000 non-null  category
dtypes: category(1)
memory usage: 68.9 KB


In [14]:
# 데이터셋 생성===> DF,NP
my_dataset=DLDataset(featureDF,targetDF.astype('float'))
my_dataset[0],featureDF.iloc[0],targetDF.iloc[0]

[target & target SHAPE] torch.Size([70000, 1]) 2


((tensor([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            1.,   0.,   0.,  13.,  73.,   0.,   0.,   1.,   4.,   0.,   0.,   0.,
            0.,   1.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   3.,   0.,  36., 136., 127.,  62.,  54.,   0.,
            0.,   0.,   1.,   3.,   4.,   0.,   0.,   3.,   0.,   0.,   0.,   0.,
            0., 

### [2-3] 학습용, 검증용, 테스트용(3개로 나누기) Dataset <hr> 

In [15]:
from torch.utils.data import random_split

#학습용,검증용,테스트 데이터 비율
seed=torch.Generator().manual_seed(42)
trainDS,validDS,testDS=random_split(my_dataset,[0.7,0.1,0.2],generator=seed) # 비율로 주기
#      학습용                           검증용                      테스트용
print(f'trainDS=>{len(trainDS)}개,validDS={len(validDS)}개,testDS={len(testDS)}개')
print('--------------------------------------------------------------------------')
print(f'Subset 속성 => indices : {trainDS.indices} \ndataset : {trainDS.dataset}')
print('-------------------------------------------------------------------------')
print(f'Subset 속성 => indices : {validDS.indices} \ndataset : {validDS.dataset}')
print('-------------------------------------------------------------------------')
print(f'Subset 속성 => indices : {testDS.indices} \ndataset : {testDS.dataset}')

trainDS=>49000개,validDS=7000개,testDS=14000개
--------------------------------------------------------------------------
Subset 속성 => indices : [37542, 64941, 23546, 30182, 19690, 46550, 59030, 28299, 61542, 23344, 47108, 6510, 29990, 15346, 22429, 44772, 59571, 67351, 10793, 13718, 25029, 18599, 10733, 356, 30155, 3413, 18425, 56065, 44795, 2464, 64112, 27904, 58107, 49380, 66975, 54754, 24351, 54768, 39034, 26473, 50706, 36331, 1307, 757, 20935, 58448, 36867, 47018, 9837, 14313, 15620, 54179, 29942, 26444, 5244, 15179, 48383, 25175, 28452, 41580, 63806, 57088, 4866, 2581, 214, 45536, 28539, 52742, 40819, 41585, 15747, 62001, 23304, 46166, 12548, 5134, 62611, 56492, 29444, 10200, 29974, 50842, 2994, 53092, 35074, 825, 35521, 2703, 60579, 44477, 34797, 55584, 50757, 59842, 38723, 66365, 27497, 37796, 66945, 139, 41870, 32808, 68295, 5886, 21241, 36931, 62827, 52515, 58981, 40975, 11525, 19482, 21559, 9775, 33120, 17289, 42200, 4087, 62521, 2201, 62919, 33329, 10055, 325, 56936, 35329, 46

[3] DataLoader 생성 : 학습용,검증용,테스트용 <hr> 

In [16]:
# trainDS.info()

In [17]:
#DataLoader 생성
# drop_last 매개변수: 배치사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본: False]
batch=1000
trainDL=DataLoader(trainDS,batch_size=batch)
validDL=DataLoader(validDS,batch_size=batch)
testDL=DataLoader(testDS,batch_size=batch)

len(trainDL),len(validDL),len(testDL)  #105/batch   , 15/batch  ,30/batch 

(49, 7, 14)

In [18]:
# Epoch 당 반복 단위
print(f'batch_size:{batch}')
print(f'trainDS=>{len(trainDS)}개,validDF=>{len(validDS)}개,testDS=>{len(testDS)}개')
print(f'trainDL=>{len(trainDL)}개,validDL=>{len(validDL)}개,testDL=>{len(testDL)}개')

batch_size:1000
trainDS=>49000개,validDF=>7000개,testDS=>14000개
trainDL=>49개,validDL=>7개,testDL=>14개


In [19]:
# DataLoader 속성
for _,(feature,target) in enumerate(trainDL):
    print(f'[{_} feature{feature.shape}')
    ## 로더에서 가지고온 데이터 만큼 학습 진행 => 검증 진행

[0 featuretorch.Size([1000, 784])
[1 featuretorch.Size([1000, 784])
[2 featuretorch.Size([1000, 784])
[3 featuretorch.Size([1000, 784])
[4 featuretorch.Size([1000, 784])
[5 featuretorch.Size([1000, 784])
[6 featuretorch.Size([1000, 784])
[7 featuretorch.Size([1000, 784])
[8 featuretorch.Size([1000, 784])
[9 featuretorch.Size([1000, 784])
[10 featuretorch.Size([1000, 784])
[11 featuretorch.Size([1000, 784])
[12 featuretorch.Size([1000, 784])
[13 featuretorch.Size([1000, 784])
[14 featuretorch.Size([1000, 784])
[15 featuretorch.Size([1000, 784])
[16 featuretorch.Size([1000, 784])
[17 featuretorch.Size([1000, 784])
[18 featuretorch.Size([1000, 784])
[19 featuretorch.Size([1000, 784])
[20 featuretorch.Size([1000, 784])
[21 featuretorch.Size([1000, 784])
[22 featuretorch.Size([1000, 784])
[23 featuretorch.Size([1000, 784])
[24 featuretorch.Size([1000, 784])
[25 featuretorch.Size([1000, 784])
[26 featuretorch.Size([1000, 784])
[27 featuretorch.Size([1000, 784])
[28 featuretorch.Size([1000, 7

[4] Model 클래스 정의: <hr> : 입/출력 피쳐수,층 수,은닉층의 노드수   <hr>
-구조 설계 
    * 입력층 :입력 <=피쳐 갯수,fashion 개
    * 은닉층: 마음대로 알아서 잘
    * 출력층: 출력 <=[분류]타겟 클래스 갯수[타겟으로 정할 열의 고유값 개수] [회귀]개

In [20]:
target.unique()

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [21]:
#모델 클래스 정의
#클래스명:CModel
class CModel(nn.Module):
    #구성 요소 정의 함수
    def __init__(self,in_dim=784,out_dim=10):
        super().__init__()
        self.input_layer=nn.Linear(in_dim,300) #1  피쳐 개수 4개 주입
        self.relu=nn.ReLU()
        self.hidden_layer=nn.Linear(300,81)  #2
        self.output_layer=nn.Linear(81,out_dim)    #3  마지막에 꽃종류 3개 반출
        
    #순방향 학습 진행 함수
    def forward(self,x):
        y=self.input_layer(x)   #W1X1+W2X2+....+Wnxn+b 반환(#1:100개)를 반환,아직 activation function을 덜 넣음
        y=self.relu(y)            #relu 함수 결과 100개 반환
        y=self.hidden_layer(y)      #W1X1+W2X2+....+Wnxn+b 반환(#2:27개)를 반환
        y=self.relu(y)
        y=self.output_layer(y)    #W1X1+W2X2+....+Wnxn+b 반환(#3:3개)를 반환
        return y
        

### [5] 학습을 위한 준비:실행디바이스,모델, 최적화, 손실함수, 학습 횟수,학습 함수,평가함수,예측함수  <hr> 

[5-1] 실행 디바이스 설정

In [22]:
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'  #얘를 상수화 하려면 튜플(tuple)로 만들어서 표현한다.

#학습 횟수
EPOCHS=50

In [23]:
my_dataset.feature.shape

torch.Size([70000, 784])

In [24]:
my_dataset.feature.shape[1]

784

In [25]:
#모델 인스턴스
IN,OUT=my_dataset.feature.shape[1],len(np.unique(targetDF))
model=CModel(IN,OUT).to(DEVICE)
print(f'IN : {IN},OUT :{OUT}')
print(model)

IN : 784,OUT :10
CModel(
  (input_layer): Linear(in_features=784, out_features=300, bias=True)
  (relu): ReLU()
  (hidden_layer): Linear(in_features=300, out_features=81, bias=True)
  (output_layer): Linear(in_features=81, out_features=10, bias=True)
)


In [26]:
# 손실함수
LOSS_FN=nn.CrossEntropyLoss().to(DEVICE)
# 최적화
import torch.optim as optim
optimizer=optim.Adam(model.parameters()) #여기서 lr 줄수 있음

In [27]:
### 학습 및 검증 관련 함수 정의

In [28]:
# import torchmetrics.functional as metrics
# 
# 
# 
# acc=metrics.accuracy(preds,target,task='multiclass',num_classes=5)
# print(f'ACC:{acc}')

[5-2] 학습진행 함수

In [29]:
import torchmetrics.functional as metrics

In [30]:
def training():
    #학습 모드 => 정규화,경사하강법,드랍아웃등의 기능을 활성화
    model.train() #모델 학습 신호
    #배치크기 만큼 학습 진행 및 저장
    train_loss=[]
    for cnt,(feature,target) in enumerate(trainDL):
        #배치크기만큼의 학습 데이터 준비
        feature,target=feature.to(DEVICE),target.to(DEVICE)
        target=target.squeeze()  #차원 축소
        
        #학습
        pre_target=model(feature)
        print(f'pre_target => {pre_target.shape},{pre_target.ndim}D')
        print(f'target => {target.shape},{target.ndim}D')
        #손실계산(지금 필요 X)
        loss=LOSS_FN(pre_target,target)
        train_loss.append(loss)
   
        #w,b 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #배치 단위 학습 진행 메시지 출력
        # print(f'[Train {cnt} batch LOSS] ==> {loss}')
        
    # 에포크 단위 진행 메시지 출력
    acc=metrics.accuracy(pre_target,target,task='multiclass',num_classes=10)  #num_classes 이거는 10
    print(f'[Train loss] ==> {loss} [ACC] ==> {acc} \n\n')
    
    return train_loss

In [31]:
for cnt,(feature,target) in enumerate(trainDL):
        print(cnt,feature,target)

0 tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ..., 13., 60.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]) tensor([[9],
        [6],
        [5],
        [3],
        [8],
        [7],
        [9],
        [7],
        [1],
        [4],
        [2],
        [7],
        [6],
        [7],
        [1],
        [5],
        [7],
        [8],
        [1],
        [7],
        [3],
        [5],
        [6],
        [8],
        [5],
        [6],
        [6],
        [5],
        [4],
        [4],
        [2],
        [7],
        [5],
        [9],
        [8],
        [7],
        [4],
        [7],
        [3],
        [0],
        [0],
        [7],
        [5],
        [3],
        [7],
        [5],
        [5],
        [0],
        [9],
        [0],
        [1],
        [4],
        [5],
        [2],
      

[5-3] 검증 및 평가 진행함수

In [32]:
# 매개변수 dataLoader : 검증 또는 테스트 데이터셋에 대한 Loader
def testing(dataLoader):
    #추론 모드 => 정규화,경사하강법,드랍아웃등의 기능을 비활성화
    model.eval()
    #배치크기 만큼 학습 진행 및 저장
    with torch.no_grad():
        #배치크기 만큼 학습 진행 및 저장
        val_loss=[]
        for cnt,(feature,target) in enumerate(dataLoader):
            #배치크기만큼의 학습 데이터 준비
            feature,target=feature.to(DEVICE),target.to(DEVICE)
            target=target.squeeze()  #차원 축소
            
            #학습
            pre_target=model(feature)
            print(f'pre_target => {pre_target.shape},{pre_target.ndim}D')
            print(f'target => {target.shape},{target.ndim}D')
            #손실계산
            loss=LOSS_FN(pre_target,target)
            val_loss.append(loss)
       
            
            
            
        # 에포크 단위 진행 메시지 출력
        acc=metrics.accuracy(pre_target,target,task='multiclass',num_classes=10)  #num_classes 이거는 10
        print(f'[Valid loss] ==> {loss} [Valid loss] ==> {acc} \n\n')
        
        return val_loss

[5-4] 예측 함수

In [33]:
def predict():
    pass

[6] 학습 진행 <hr>

In [34]:
### ===>지정된 횟수 만큼 처음부터~ 끝까지 학습 및 검증 진행
### ===>목표:최적(Error 최소화)의 W,b를 가진 모델 완성
### ===>
for eps in range(1,EPOCHS+1):
    # 학습 함수
    train_loss=training()
    #검증 함수
    valid_loss=testing(validDL)  #validDL 넣어야 돌아간다
    print(f'[{eps}/{EPOCHS}]{sum(train_loss)/len(train_loss)},\
          VALID:{sum(valid_loss)/len(valid_loss)}\n')
    #조기종료 기능 ==> 조건 : val_loss가 지정된 횟수(예:5)이상 개선이 안되면 학습 종료

pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([1000]),1D
pre_target => torch.Size([1000, 10]),2D
target => torch.Size([10

[7] 테스트 진행 <hr>